In [25]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.animation import FuncAnimation
from numpy.fft import fft, ifft, fftfreq
import seaborn as sns
from nltools.data import Brain_Data, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import isc, isfc, isps, fdr, threshold, phase_randomize, circle_shift, _butter_bandpass_filter, _phase_mean_angle, _phase_vector_length
from nilearn.plotting import view_img_on_surf, view_img
from sklearn.metrics import pairwise_distances
from sklearn.utils import check_random_state
from scipy.stats import ttest_1samp
from scipy.signal import hilbert
import networkx as nx
from IPython.display import HTML

In [3]:
basepath = '/Users/yana/OneDrive - University of Wyoming/RESEARCH/BBC'
filefolders = '/Users/yana/OneDrive - University of Wyoming/RESEARCH/BBC/data'
baseline_path = '/Users/yana/OneDrive - University of Wyoming/RESEARCH/BBC/data/base/'

In [4]:
os.path.exists(baseline_path)


True

In [31]:
df = pd.DataFrame()
#sub_number = 0
#file = pd.read_csv(os.path.join(baseline_path, '20001', 'total_ts.csv'))
#time_column = file[['time']]
#df = pd.concat([df, time_column], axis = 1)

for filepath in glob.glob(os.path.join(baseline_path,'*/total_ts.csv')): 
    #print(filepath)
    file = pd.read_csv(filepath)
    #time_column = file[['time']]
    file = file[['Pu']]    
    sub_num = filepath.split('/')[-2]
    sub_num = sub_num[3:5]
    file.rename(columns = {'Pu':('sub-'+sub_num)}, inplace = True)
    df = pd.concat([df, file], axis = 1)

    print(sub_num)
    
file

#sub_number = [time, sub-01, sub-02, sub-03, sub-04, sub-08, sub-10] 

 #sub_number = sub_number + 1
    #data = file
    #df = pd.DataFrame(data, columns=['sub_'+str(sub_number)])
    
    
print(df)
    #file = file.transpose()

08
01
02
03
04
10
     sub-08    sub-01    sub-02    sub-03    sub-04    sub-10
0 -0.644472  1.797876 -0.815184 -0.376934  0.141275 -0.351786
1 -0.770741  0.639720 -0.549698 -0.860777  0.180017 -0.641288
2 -0.364052  0.355728 -0.087650 -0.734210 -0.298075 -0.208653
3 -0.148369  0.194052 -0.423381 -0.298679  0.415681 -0.127131
4 -0.041893 -0.128680  0.162000 -0.270529 -0.252525 -0.357007
5  0.163598 -0.166101  0.307607 -0.000120  0.130586 -0.079464
6  0.185239 -0.158817  0.257239  0.534270  0.021947 -0.132999
7  0.494937 -0.747150  0.288180  0.451536  0.203497  0.748300
8  0.292030 -0.880075  0.438539  0.821202 -0.374129  0.630110
9  0.833724 -0.906553  0.422347  0.734240 -0.168275  0.519918


In [ ]:
sub = 'sub-08'
sampling_freq = .5

f,a = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
a[0,0].plot(df[sub], linewidth=2)
a[0,0].set_ylabel('Avg Activity', fontsize=16)
a[0,1].set_xlabel('Time (TR)', fontsize=18)
a[0,0].set_title('Observed Data', fontsize=16)

fft_data = fft(df[sub])
freq = fftfreq(len(fft_data), 1/sampling_freq)
n_freq = int(np.floor(len(fft_data)/2))
a[0,1].plot(freq[:n_freq], np.abs(fft_data)[:n_freq], linewidth=2)
a[0,1].set_xlabel('Frequency (Hz)', fontsize=18)
a[0,1].set_ylabel('Amplitude', fontsize=18)
a[0,1].set_title('Power Spectrum', fontsize=18)

circle_shift_data = circle_shift(df[sub])
a[1,0].plot(circle_shift_data, linewidth=2, color='red')
a[1,0].set_ylabel('Avg Activity', fontsize=16)
a[1,0].set_xlabel('Time (TR)', fontsize=16)
a[1,0].set_title('Circle Shifted Data', fontsize=16)

fft_circle = fft(circle_shift_data)
a[1,1].plot(freq[:n_freq], np.abs(fft_circle)[:n_freq], linewidth=2, color='red')
a[1,1].set_xlabel('Frequency (Hz)', fontsize=18)
a[1,1].set_ylabel('Amplitude', fontsize=18)
a[1,1].set_title('Circle Shifted Power Spectrum', fontsize=18)

plt.tight_layout()

In [30]:
stats_circle = isc(df, method='circle_shift', n_bootstraps=5000, return_bootstraps=True)

print(f"ISC: {stats_circle['isc']:.02}, p = {stats_circle['p']:.03}")


ISC: -0.23, p = 0.0314
